# Harvester GetValueRequest
This is a KWH HMI which allows to request the harvester's data. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to call up the harvester's data. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [37]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests
from IPython.display import Image
from tools import print_with_timestamp, check_message_encryption, yes, no

## Configure the notebook
In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [38]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
hmiId = input('[S3I]: Please enter your HMI id:').strip('," ')
hmiSecret = getpass.getpass('[S3I]: Please enter your HMI secret:').strip('," ')
print_with_timestamp("HMI id and secret are set")

[S3I][2020-07-27 21:36:56]: Assign a client to this notebook. (The id of your HMI)
[S3I]: Please enter your HMI id:s3i:658691b2-748f-4066-8945-fd4b98a1c058
[S3I]: Please enter your HMI secret:········
[S3I][2020-07-27 21:37:12]: HMI id and secret are set


Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the harvester's data on your behalf.

In [39]:
print_with_timestamp("DEMO harvester, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmiId,
                                             client_secret=hmiSecret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

[S3I][2020-07-27 21:37:12]: DEMO harvester, please log in!
[S3I]: Please enter your username:user3
[S3I]: Please enter the password:········
[S3I][2020-07-27 21:37:18]: Your credentials are sent to S3I IdentityProvider.
[S3I][2020-07-27 21:37:19]: Token received, user3 logged in.


After the token has been received, it can be used to query the endpoint of the harvester. To query the harvester in the S³I Directory, you must have the rights to view the harvester. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the harvester, the enpoint of this HMI is also needed to add as "ReplyToEnpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may has changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [40]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print_with_timestamp("Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoints = s3i_directory.queryThingIDBased(hmiId+"/attributes/allEndpoints")
for item in sender_endpoints:
    if "s3i:" in item:
        sender_endpoint = item
    else:
        continue
    
''' query the harvesters's Id and endpoint
'''
harvesterID = s3i_directory.queryAttributeBased(
    "name", "gSFL-Harvester")[0]["thingId"]

harvesterEndpoint = s3i_directory.queryThingIDBased(harvesterID+"/attributes/allEndpoints")
receivers = [harvesterID]
receiver_endpoints = harvesterEndpoint

[S3I][2020-07-27 21:37:19]: Authentication with Token in S3I Directory successful


## MessageId Storage
The messageId storage stores all message ids from the messages which are send. If a message arrives it is checked against these ids, to see if it is a response to one of these messages. If it is not a response to one a request send from this notebook, the message won't be parsed.

In [41]:
messageIds = list()

# Get Harvester S³I Directory Entry

In [42]:
%%html
<img src="harvester_images/dir.png", width=500, height=500>

In [43]:
send_dir_rest = input("[S3I] Do you want to send a rest request to query the harvester in S3I-Directory? [j/n]")
if send_dir_rest in yes:
    harvesterDir = s3i_directory.queryThingIDBased(harvesterID)
    print_with_timestamp(json.dumps(harvesterDir, indent=2))
elif send_dir_rest in no:
        print_with_timestamp("You do not want to send a request to query the harvester in S3I-Directory.")
else:
        print_with_timestamp("I could not understand your response. If you want to send a rest request again, just execute this cell again (Run button or SHIFT+RETURN)")    

[S3I] Do you want to send a rest request to query the harvester in S3I-Directory? [j/n]j
[S3I][2020-07-27 21:37:23]: {
  "thingId": "s3i:ef39a0ae-1f4a-4393-9508-ad70a4d38a63",
  "policyId": "s3i:ef39a0ae-1f4a-4393-9508-ad70a4d38a63",
  "attributes": {
    "ownedBy": "...",
    "name": "gSFL-Harvester",
    "location": {
      "longitude": 7.995628567992779,
      "latitude": 51.45290905110912
    },
    "type": "component",
    "dataModel": "fml40",
    "allEndpoints": [
      "s3ib://s3i:ef39a0ae-1f4a-4393-9508-ad70a4d38a63",
      "https://ditto.s3i.vswf.dev/api/2/things/s3i:ef39a0ae-1f4a-4393-9508-ad70a4d38a63"
    ],
    "thingStructure": {
      "class": "ml40::Thing",
      "links": [
        {
          "association": "roles",
          "target": {
            "class": "fml40::Harvester"
          }
        },
        {
          "association": "features",
          "target": {
            "class": "fml40::ProvidesProductionData"
          }
        },
        {
          "assoc

# Get Harvester Cloud Copy from the S³I Repository

In [ ]:
%%html
<img src="harvester_images/repo.png", width=500, height=500>

In [ ]:
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
send_repo_rest = input("[S3I] Do you want to send a rest request to query the harvester in S3I-Repository? [j/n]")
if send_repo_rest in yes:
    response = requests.get(
        url="https://ditto.s3i.vswf.dev/api/2/things/"+harvesterID, headers=headers)
    print_with_timestamp(json.dumps(response.json(), indent=2))
elif send_repo_rest in no:
        print_with_timestamp("You do not want to send a request to query the harvester in S3I-Repository.")
else:
        print_with_timestamp("I could not understand your response. If you want to send a rest request again, just execute this cell again (Run button or SHIFT+RETURN)")    

# >>> *Run All below* from here to request another attribute
## Prepared the request
After querying the enpoints, the message is prepared. A *GetValueRequest* is instantiated and filled in.

In [ ]:
%%html
<img src="harvester_images/edge.png", width=500, height=500>

In [ ]:
Image(filename="harvester_images/edge.png", width=100, height=100)
print_with_timestamp("Prepare the GetValueRequest.")
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the harvester. Examples: \nattributes/features/ml40::Composite/targets/ml40::Engine/features/ml40::RotationalSpeed/rpm \n').strip('," ')

msg_uuid = "s3i:" + str(uuid.uuid4())

servReq = s3i.GetValueRequest()
servReq.fillGetValueRequest(senderUUID=hmiId, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                                attributePath=attributePath, msgUUID=msg_uuid)
print_with_timestamp("GetValueRequest prepared:")
print(json.dumps(servReq.msg, indent=2))

## Send the request
The HMI requests a new token to establish a connection to the broker and sends the request to the harvester's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the GetValueRequest to the harvester")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
messageIds.append(servReq.msg["identifier"])
for item in receiver_endpoints:
    if "s3ib" in item and "https" not in item:
        s3ib_endpoint = item
    else:
        continue 
response = requests.post(url="https://broker.s3i.vswf.dev/"+s3ib_endpoint,
                                data=json.dumps(servReq.msg), headers=headers)
print_with_timestamp(response.text)

## Receive the harvester's response

In [ ]:
def receive():
    print_with_timestamp("Checking the harvester's response")
    access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
    headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
    response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
    json_acceptable_string = response.text.replace("'", "\"")

    if json_acceptable_string:
        if check_message_encryption(response.text.strip('"')) == "pgp":
            print_with_timestamp("You received a PGP message but this notebook can not decrypt PGP messages. Use the 03_inbox notebook to receive PGP messages.")
            print_with_timestamp("PGP Message: " + response.text)
            repeat()
        else: 
            response_json = json.loads(json_acceptable_string)
            print_with_timestamp("You received a message: " + json.dumps(response_json, indent=2))
            if response_json["replyingToMessage"] in messageIds:
                print_with_timestamp("It is the respose from the harvester concerning your GetValueRequest with the id " + response_json["replyingToMessage"] + ". Your requested value is: " + json.dumps(response_json["value"], indent=2))
                messageIds.remove(response_json["replyingToMessage"])
    else:
        print_with_timestamp("The harvester did not respond yet.")
        repeat()


def repeat():
    decision = input("[S3I] Do you want to check for new messages again? [j/n]")
    if decision in yes:
        receive()
    elif decision in no:
        print_with_timestamp("You do not want to check for more messages. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")
    else:
        print_with_timestamp("I could not understand your response. If you want to check for new messages, just execute this cell again (Run button or SHIFT+RETURN)")

In [ ]:
receive()